In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.4)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.80 sec - Loss 0.062100 - ACC 57.25% - ACC Mean 57.25% - AUC 60.00% - AUC Mean 60.00% - Deter 000
Ite 00050 - 1.69 sec - Loss 0.011730 - ACC 56.55% - ACC Mean 57.80% - AUC 58.36% - AUC Mean 60.83% - Deter 003
Ite 00097 - 1.67 sec - Loss 0.006119 - ACC 58.10% - ACC Mean 58.03% - AUC 62.10% - AUC Mean 61.21% - Deter 050
Early stop ite 97, rollback to correction of ite 47, whith acc of 59.4% and auc of 63.62%
OUR METHOD RUN 1/10 - acc: 59.4% +- 0.0% - auc: 63.62% +- 0.0%
Ite 00000 - 1.42 sec - Loss 0.125068 - ACC 59.50% - ACC Mean 59.50% - AUC 63.05% - AUC Mean 63.05% - Deter 000
Ite 00050 - 1.72 sec - Loss 0.045110 - ACC 58.10% - ACC Mean 58.07% - AUC 61.01% - AUC Mean 61.29% - Deter 013
Ite 00087 - 2.03 sec - Loss 0.029097 - ACC 57.55% - ACC Mean 57.96% - AUC 60.08% - AUC Mean 61.27% - Deter 050
Early stop ite 87, rollback to correction of ite 37, whith acc of 58.85% and auc of 64.18%
OUR METHOD RUN 2/10 - acc: 58.85% +- 0.0% - auc: 64.18% +- 0.0%
Ite 00000 - 2.44 sec - Los

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 57.6% +- 0.0% - auc: 61.31% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 56.3% +- 1.61% - auc: 60.0% +- 3.22%
SFIL RUN 2/10 - acc: 53.65% +- 2.4% - auc: 56.41% +- 1.12%
SFIL RUN 3/10 - acc: 54.4% +- 2.33% - auc: 57.79% +- 3.21%
SFIL RUN 4/10 - acc: 53.55% +- 2.39% - auc: 55.3% +- 2.44%
SFIL RUN 5/10 - acc: 54.65% +- 2.71% - auc: 58.34% +- 2.53%
SFIL RUN 6/10 - acc: 55.55% +- 3.02% - auc: 58.53% +- 2.17%
SFIL RUN 7/10 - acc: 54.65% +- 2.82% - auc: 58.39% +- 3.98%
SFIL RUN 8/10 - acc: 54.7% +- 1.46% - auc: 56.87% +- 2.73%
SFIL RUN 9/10 - acc: 53.75% +- 2.2% - auc: 57.55% +- 2.73%
SFIL RUN 10/10 - acc: 53.85% +- 1.45% - auc: 56.21% +- 2.3%
SFIL GLOBAL - acc: 54.5% +- 0.84% - auc: 57.54% +- 1.3%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 58.4% +- 2.04% - auc: 61.77% +- 3.54%
PFIL RUN 2/10 - acc: 58.55% +- 1.9% - auc: 61.93% +- 1.45%
PFIL RUN 3/10 - acc: 58.45% +- 0.95% - auc: 61.88% +- 0.88%
PFIL RUN 4/10 - acc: 58.55% +- 2.57% - auc: 62.35% +- 3.13%
PFIL RUN 5/10 - acc: 59.7% +- 2.18% - auc: 62.81% +- 3.18%
PFIL RUN 6/10 - acc: 57.6% +- 1.79% - auc: 59.68% +- 2.01%
PFIL RUN 7/10 - acc: 59.25% +- 1.06% - auc: 62.2% +- 1.7%
PFIL RUN 8/10 - acc: 60.4% +- 2.94% - auc: 63.71% +- 2.72%
PFIL RUN 9/10 - acc: 57.7% +- 2.44% - auc: 61.06% +- 2.18%
PFIL RUN 10/10 - acc: 59.7% +- 1.44% - auc: 62.63% +- 1.94%
PFIL GLOBAL - acc: 58.83% +- 0.86% - auc: 62.0% +- 1.02%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 53.8% +- 3.1% - auc: 56.98% +- 4.29%
SPOL RUN 2/10 - acc: 54.1% +- 1.11% - auc: 56.21% +- 1.83%
SPOL RUN 3/10 - acc: 54.1% +- 3.76% - auc: 55.74% +- 3.84%
SPOL RUN 4/10 - acc: 54.6% +- 1.63% - auc: 56.92% +- 2.02%
SPOL RUN 5/10 - acc: 54.1% +- 2.42% - auc: 56.86% +- 2.89%
SPOL RUN 6/10 - acc: 54.3% +- 2.58% - auc: 55.76% +- 3.09%
SPOL RUN 7/10 - acc: 54.9% +- 1.9% - auc: 57.08% +- 2.53%
SPOL RUN 8/10 - acc: 53.4% +- 1.35% - auc: 55.85% +- 1.86%
SPOL RUN 9/10 - acc: 54.45% +- 0.66% - auc: 57.43% +- 1.07%
SPOL RUN 10/10 - acc: 53.45% +- 2.26% - auc: 55.65% +- 2.59%
SPOL GLOBAL - acc: 54.12% +- 0.45% - auc: 56.45% +- 0.64%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 60.05% +- 1.28% - auc: 61.49% +- 1.9%
PPOL RUN 2/10 - acc: 60.95% +- 3.28% - auc: 63.36% +- 3.28%
PPOL RUN 3/10 - acc: 58.4% +- 1.37% - auc: 61.63% +- 1.89%
PPOL RUN 4/10 - acc: 59.85% +- 2.04% - auc: 62.79% +- 1.92%
PPOL RUN 5/10 - acc: 58.3% +- 2.18% - auc: 60.96% +- 2.41%
PPOL RUN 6/10 - acc: 58.4% +- 3.8% - auc: 61.63% +- 3.73%
PPOL RUN 7/10 - acc: 59.45% +- 2.61% - auc: 63.37% +- 2.48%
PPOL RUN 8/10 - acc: 60.4% +- 2.13% - auc: 63.9% +- 2.39%
PPOL RUN 9/10 - acc: 57.9% +- 2.21% - auc: 61.94% +- 1.87%
PPOL RUN 10/10 - acc: 58.2% +- 1.44% - auc: 61.96% +- 1.13%
PPOL GLOBAL - acc: 59.19% +- 1.02% - auc: 62.3% +- 0.93%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is even with PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
